# speech-correction 모듈 테스트

## 패키지 import 사전 설정

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

rvc_path = os.path.abspath(os.path.join('../Retrieval-based-Voice-Conversion-WebUI-main'))
if rvc_path not in sys.path:
    sys.path.append(rvc_path)

In [ ]:
# ToDo: 의존 패키지 목록 작성 (미완료)
# STT 의존 패키지
%pip install openai-whisper
%pip install torch torchvision torchaudio
# ffmpeg 설치 필요

# Correction
%pip install python-dotenv

### 공통 상수/변수 설정

In [ ]:
# STT
ORIGIN_VOICE_PATH = "demo-file/test.wav"
ORIGIN_SCRIPT_TXT_PATH = "demo-file/origin-script.txt"
origin_script_str = "TTS로 변환된 스크립트"

# Correction
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CORRECTED_SCRIPT_TXT_PATH = "demo-file/corrected-script.txt"
corrected_script_str = "chatGPT로 교정된 스크립트"

# TTS
MODEL_VOICE_PATH = "demo-file/speaker_voice.wav"
BASIC_VOICE_PATH = "demo-file/basic-voice.wav"

# Voice
CHANGED_VOICE_PATH = "demo-file/changed-voice.wav"

---
## STT 모듈 테스트

In [ ]:
# 모듈 import 및 객체 생성
from speechcorrection import stt

stt = stt.ScriptGenerator()

In [ ]:
# 실행
stt.origin_voice_path = ORIGIN_VOICE_PATH
stt.execute()

origin_script_str = stt.origin_script
print(stt.origin_script)

with open(ORIGIN_SCRIPT_TXT_PATH, "w") as file:
    file.write(origin_script_str)

---
## Correction 모듈 테스트

In [ ]:
# 모듈 import 및 객체 생성
from speechcorrection import correction

correction = correction.ScriptCorrection()
correction.api_key = OPENAI_API_KEY

In [ ]:
# 파일로 원본 스크립트 가져오기 (위에서 STT 동작했으면 실행 생략 가능)
with open(ORIGIN_SCRIPT_TXT_PATH, "r") as file:
    origin_script_str = file.read()
print(origin_script_str)

In [ ]:
# 실행
correction.origin_script = origin_script_str
correction.execute()

corrected_script_str = correction.corrected_script
print(corrected_script_str)

with open(CORRECTED_SCRIPT_TXT_PATH, "w") as file:
    file.write(corrected_script_str)

---
## TTS 모듈 테스트

In [ ]:
# 모듈 import 및 객체 생성
from speechcorrection import tts

tts = tts.VoiceGenerator()

In [ ]:
# 파일로 교정된 스크립트 가져오기 (위에서 Correction 동작했으면 실행 생략 가능)
with open(CORRECTED_SCRIPT_TXT_PATH, "r") as file:
    corrected_script_str = file.read()
print(corrected_script_str)

In [ ]:
tts.corrected_script = corrected_script_str
tts.basic_voice_path = MODEL_VOICE_PATH
tts.output_wav_path = BASIC_VOICE_PATH

tts.execute()



[speaker_voice.wav](./demo-file/speaker_voice.wav)

---
# Voice 모듈 테스트

In [ ]:
from speechcorrection import voice
from configs.config import Config

config = Config()

voice_changer = voice.VoiceChanger(config)
voice_changer.basic_voice_path = "rvc_test_origin.wav"
now_dir = os.getcwd()
voice_changer.changed_voice_path = os.path.join(
    now_dir, "assets", "vc_output", "vc_output.wav"
)
voice_changer.load_vc_model = "IU"

voice_changer.execute()